In [266]:
# session.write_pandas(df,table_name="HR_CHURN",auto_create_table=True).show()
# !pip install fosforml

In [6]:
from fosforml.model_manager.snowflakesession import get_session

In [7]:
session = get_session()
session.get_current_schema().replace('"','')


'INSIGHT_SPCS_SCHEMA'

In [13]:
from snowflake.snowpark.session import Session,col
import snowflake.snowpark.functions
from snowflake.ml.modeling.metrics import confusion_matrix, accuracy_score, f1_score, recall_score,precision_score
from snowflake.ml.modeling.preprocessing import LabelEncoder, StandardScaler
from snowflake.ml.modeling.impute import SimpleImputer
from snowflake.ml.modeling.ensemble import GradientBoostingClassifier,RandomForestClassifier
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake

def apply_label_encoding(input_data_frame):
    for i in input_data_frame.dtypes:
        if i[1].find('string') >= 0:
            label_encoder = LabelEncoder(input_cols=i[0],output_cols=i[0],drop_input_cols=True)
            input_data_frame = label_encoder.fit(input_data_frame).transform(input_data_frame)
    return input_data_frame

In [14]:
import pandas as pd
df = pd.read_csv("HR_CHURN.csv")
# snow_df = session.table("HR_CHURN")
snow_df = session.create_dataframe(df)
snow_df = snow_df.drop(['Unnamed: 0'])
snow_df.limit(1).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"DEPARTMENT"  |"SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |sales         |low       |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [15]:
input_data_frame, test_df = snow_df.randomSplit([0.75, 0.25])
input_data_frame.limit(1).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"DEPARTMENT"  |"SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |sales         |medium    |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [16]:
label_encoder_column = []     
# input_data_frame = apply_label_encoding(input_data_frame)
feature_cols = input_data_frame.columns
target_col = "SALARY"
feature_cols.remove(target_col)
OUTPUT_COLS = [target_col + '_PREDICTION']
input_data_frame.limit(1).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |"DEPARTMENT"  |"SALARY"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |sales         |medium    |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [17]:

pipeline = RandomForestClassifier()
# pipeline = Pipeline(
#     steps = [
#         ("SnowflakePiplineModel",
#          RandomForestClassifier(input_cols=feature_cols, label_cols=target_col))
#     ]
# )

In [18]:
pipeline.fit(input_data_frame)

Got error object of type 'NoneType' has no len() when trying to read default values from function: <function SnowparkModelTrainer._build_fit_wrapper_sproc.<locals>.fit_wrapper_function at 0x7f736c656f70>. Proceeding without creating optional arguments
The version of package 'snowflake-snowpark-python' in the local environment is 1.21.1, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.
The version of package 'scikit-learn' in the local environment is 1.3.2, which does not fit the criteria for the requirement 'scikit-learn==1.3.0'. Your UDF might not work when the package version is different between the server and your local environment.


SnowparkSQLException: (1300) (1304): 01b6df4e-0710-dac5-0072-f30310ab563e: 100357 (P0000): Python Interpreter Error:
Traceback (most recent call last):
  File "_udf_code.py", line 77, in compute
  File "/opt/conda/lib/python3.9/site-packages/snowflake/ml/modeling/_internal/snowpark_implementations/snowpark_trainer.py", line 233, in fit_wrapper_function
  File "/usr/lib/python_udf/a56a808b073b78261b72719f8160208a2cba61861bcbe7ac5101382f8eaad75d/lib/python3.9/site-packages/sklearn/base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/lib/python_udf/a56a808b073b78261b72719f8160208a2cba61861bcbe7ac5101382f8eaad75d/lib/python3.9/site-packages/sklearn/ensemble/_forest.py", line 348, in fit
    X, y = self._validate_data(
  File "/usr/lib/python_udf/a56a808b073b78261b72719f8160208a2cba61861bcbe7ac5101382f8eaad75d/lib/python3.9/site-packages/sklearn/base.py", line 621, in _validate_data
    X, y = check_X_y(X, y, **check_params)
  File "/usr/lib/python_

In [166]:
# test_en_df = apply_label_encoding(test_df)

In [167]:
test_en_df.limit(1).show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|1.0       |7.0           |0.38                  |0.53               |2                 |157                     |3                     |0                |1       |0                        |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [168]:
pred_snow_df = pipeline.predict(test_en_df)

In [169]:
prob_snow_df = pipeline.predict_proba(test_en_df)

In [173]:
input_data_frame.show()

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |0.72                  |0.87               |5                 |223                     |5                     |0                |1       |0                        |
|1.0       |7.0           |0.37              

In [203]:
set(['low', 'medium', 'high']) == set(['medium', 'low', 'high'])

True

In [200]:
[row['SALARY'] for row  in input_data_frame.select(col('SALARY')).distinct().collect()]

['low', 'medium', 'high']

In [201]:
[col_name.replace('PREDICT_PROBA_', '').replace("'","").replace('"','') for col_name in column_names if 'PREDICT_PROBA_' in col_name]

['0.0', '1.0', '2.0']

In [252]:
from snowflake.snowpark.functions import col, try_cast, when, lit

def map_to_numeric_col(df, col_name, columns_list, mapped_col_name):
    columns_map = {col: index for index, col in enumerate(columns_list)}
    colum_categories = [row[col_name] for row in df.select(col(col_name)).distinct().collect()]
    
    if set(colum_categories) == set(columns_map.keys()):
        print('matching columns')
        mapped_col = col(col_name)
        for category, value in columns_map.items():
            mapped_col = when(col(col_name) == category, lit(value).cast('FLOAT')).otherwise(mapped_col)
        df = df.with_column(mapped_col_name, mapped_col)
    else:
        temp_cols = [col_name.replace('PREDICT_PROBA_', '').replace("'", "").replace('"', '') for col_name in columns_list if 'PREDICT_PROBA_' in col_name]
        if set(colum_categories) == set(temp_cols):
            print('additional tasks')
            print(temp_cols)
            mapped_col = col(col_name)
            # df.show()
            for category, value in columns_map.items():
                mapped_col = when(col(col_name) == category, lit(value).cast('FLOAT')).otherwise(mapped_col)
                print(mapped_col,dir(mapped_col))
            df = df.with_column(mapped_col_name, mapped_col).collect()
            df.show()
        else:
            raise Exception("Predict probability column names are not matching with the values of target/prediction columns in the dataset")
        
    return df

# Example usage
# Assuming df is your Snowflake DataFrame and you have a session established
# df = session.table("your_table_name")

# Mapping example
# mapping_data = ['A', 'B', 'C']
# df_mapped = map_to_numeric_col(df, 'categorical_column', mapping_data, 'mapped_column')

# # Show the schema to verify the new column type
# df_mapped.show()

df = input_data_frame
col_name = 'SALARY'
columns_list = ["PREDICT_PROBA_low","PREDICT_PROBA_high","PREDICT_PROBA_medium"]
mapped_col_name = 'SALARY_NUMERIC'

In [254]:
input_df = map_to_numeric_col(df,col_name,columns_list,mapped_col_name)

In [264]:
input_df.select(input_df[mapped_col_name])

In [170]:
test_df = prob_snow_df.to_pandas()

In [171]:
test_df.head()

,SALARY,DEPARTMENT,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,"""PREDICT_PROBA_0.0""","""PREDICT_PROBA_1.0""","""PREDICT_PROBA_2.0"""
0,1.0,7.0,0.10,0.77,6,247,4,0,1,0,0.001736,0.768049,0.230215
1,1.0,8.0,0.43,0.47,2,147,3,0,1,0,0.005127,0.585659,0.409214
2,1.0,2.0,0.39,0.56,2,142,3,0,1,0,0.007157,0.449269,0.543574
3,2.0,7.0,0.36,0.52,2,147,3,0,1,0,0.013509,0.411396,0.575095
4,1.0,8.0,0.70,0.89,3,183,5,0,1,0,0.005062,0.879325,0.115613


In [18]:
test_df['SALARY'].unique()

array([1., 2., 0.])

In [19]:
test_df.columns

Index(['SALARY', 'DEPARTMENT', 'SATISFACTION_LEVEL', 'LAST_EVALUATION',
       'NUMBER_PROJECT', 'AVERAGE_MONTLY_HOURS', 'TIME_SPEND_COMPANY',
       'WORK_ACCIDENT', 'LEFT', 'PROMOTION_LAST_5YEARS', '"PREDICT_PROBA_0.0"',
       '"PREDICT_PROBA_1.0"', '"PREDICT_PROBA_2.0"'],
      dtype='object')

In [21]:
# dir(pipeline)

In [53]:
df = df.drop(['Unnamed: 0','DEPARTMENT'],axis=1).dropna()

In [61]:
from snowflake.ensemble._forest import RandomForestClassifier
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'snowflake.ensemble'

In [55]:
model = RandomForestClassifier()
x_train,x_test,y_train,y_test = train_test_split(df.drop('SALARY',axis=1),df['SALARY'])

In [56]:
x_train.columns

Index(['SATISFACTION_LEVEL', 'LAST_EVALUATION', 'NUMBER_PROJECT',
       'AVERAGE_MONTLY_HOURS', 'TIME_SPEND_COMPANY', 'WORK_ACCIDENT', 'LEFT',
       'PROMOTION_LAST_5YEARS'],
      dtype='object')

In [57]:
x_train.head()

,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS
2692,0.68,0.51,4,157,3,0,0,0
13580,0.86,0.84,3,177,3,0,0,0
7448,0.55,0.98,4,137,2,0,0,0
3409,0.58,0.86,5,206,3,0,0,0
570,0.44,0.52,2,137,3,0,1,0


In [58]:
model.fit(x_train,y_train)

RandomForestClassifier()

In [60]:
# model.predict_proba(x_test)

In [46]:
from fosforml import register_model

register_model(
  model_obj=pipeline,
  session=session,
  name="Testmode02092024",
  snowflake_df=pred_snow_df,
  dataset_name="HR_CHURN",
  dataset_source="Dataset",
  source="Notebook",
  description="This is a Snowflake model",
  flavour="snowflake",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"]
)

Calculating build time metrics

Progress: ██████████████                                                         20.0%
Calculating build time metrics

Progress: ████████████████████████████                                           40.0%


Calculating build time metrics

Progress: ██████████████████████████████████████████                             60.0%
Calculating build time metrics

Progress: ████████████████████████████████████████████████████████               80.0%
Calculating build time metrics

Progress: ██████████████████████████████████████████████████████████████████████ 100.0%


"Model 'MODEL_8D3FB8A6_A886_4236_9B35_73EB1303C4BF_FDC_TESTMODE02092024' registered successfully."

In [54]:
model_obj = model_registry.get_model(model_name="TestModel")

In [55]:
model = model_obj.version(version_name="v1")

In [58]:
inptu_df ,test_df = session.table("HR_CHURN").randomSplit([0.75, 0.25])

In [59]:
test_df = apply_label_encoding(test_df)

In [75]:
test_df_pandas = test_df.to_pandas()

In [76]:
session.write_pandas(test_df_pandas,table_name="TEST_DATA_HR",overwrite=True).show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [80]:
test_df_fetch = session.table("TEST_DATA_HR")

In [81]:
test_df_fetch.show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"SALARY"  |"DEPARTMENT"  |"Unnamed: 0"  |"SATISFACTION_LEVEL"  |"LAST_EVALUATION"  |"NUMBER_PROJECT"  |"AVERAGE_MONTLY_HOURS"  |"TIME_SPEND_COMPANY"  |"WORK_ACCIDENT"  |"LEFT"  |"PROMOTION_LAST_5YEARS"  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2.0       |7.0           |1             |0.8                   |0.86               |5                 |262                     |6                     |0                |1       |0                        |
|1.0       |7.0           |13            |0.41                  |0.55               |2                 |148                     |3                     |0                |1     

In [82]:
model.run(test_df_fetch, function_name="predict").to_pandas()

,SALARY,DEPARTMENT,Unnamed: 0,SATISFACTION_LEVEL,LAST_EVALUATION,NUMBER_PROJECT,AVERAGE_MONTLY_HOURS,TIME_SPEND_COMPANY,WORK_ACCIDENT,LEFT,PROMOTION_LAST_5YEARS,OUTPUT_SALARY
0,1.0,3.0,34,0.84,0.87,4,246,6,0,1,0,1.0
1,2.0,6.0,66,0.43,0.54,2,153,3,0,1,0,2.0
2,1.0,7.0,94,0.09,0.83,6,255,4,0,1,0,1.0
3,1.0,8.0,130,0.10,0.83,6,292,4,0,1,0,1.0
4,1.0,7.0,163,0.79,0.89,5,239,5,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7447,2.0,5.0,14821,0.09,0.95,7,256,4,0,1,0,2.0
7448,2.0,9.0,14859,0.45,0.54,2,129,3,0,1,0,2.0
7449,1.0,7.0,14888,0.39,0.49,2,142,3,0,1,0,1.0
7450,2.0,7.0,14945,0.14,0.75,4,277,5,1,1,0,1.0


In [ ]:
register_model(
  model_obj=pipeline,
  session=session,
  snowflake_df=pred_snow_df,
  name="HR_Churn_model",
  description="This is a test model",
  flavour="snowflake",
  dataset_name="Hr_churn",
  datasource_name="Hr_churn",
  model_type="classification",
  conda_dependencies=["scikit-learn==1.3.2"],
  source="Notebook"
)